In [3]:
import numpy as np
import random
from scipy import ndimage, misc
import copy
import time
from deap import base
from deap import creator
from deap import tools
import game


In [4]:
n_weights = 9
population_size = 20
crossover_probability = 0.5
mutation_probability = 0.1

moves = [0,1,2,3]
directions = {
        0: "down",
        1: "up",
        2: "left",
        3: "right"
    }

gameIterations = 1000

In [5]:
''' DEAP SETUP PT 1'''

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("GameInstance", game.Game)
creator.create("Individual", list, 
                fitness=creator.FitnessMax, 
                gameInstance=creator.GameInstance)

toolbox = base.Toolbox()
toolbox.register("attr_bool", np.random.uniform, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, n_weights)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [6]:
''' Example '''

# Create individual
ind = toolbox.individual()
print(ind) # 10 weights between 0-1

# Game within Individual
print("---")
print(ind.gameInstance.grid)

# Make a move in the game
ind.gameInstance.move(0) # 0 => down 
print("---")
print(ind.gameInstance.grid)

[0.008127444737063061, 0.20615117780367154, 0.8905466167192019, 0.15717078616172186, 0.7527857817509143, 0.5562483734705322, 0.0688559204193252, 0.4073782251626674, 0.25848249524642675]
---
[[0 0 0 0]
 [0 0 2 0]
 [0 0 2 0]
 [0 0 0 0]]
---
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 2]
 [0 0 4 0]]


In [90]:
''' F1 '''
# f1 : Feature f1 is 1, if tile with maximum value is on one of the corners in a 
# given state. Otherwise, it is 0.
def f1(newstate):
    flat_grid = newstate.flatten()

    maxvalue = max(flat_grid)
    for i,value in enumerate(flat_grid):
        if value == maxvalue and i in [0,3,11,15]:
            return 1
    return 0

''' F2 '''
# Feature f2 is 1, if second or third highest value tile is next to highest
# value tile. Otherwise, it is 0.
def f2andf3andf4(currentstate, newstate):
    f2 = 0
    f3 = 0
    f4 = 0
    maxval = max(newstate.flatten())
    if maxval==2: f2 = 0

    if maxval==4:
        maxvalues = [4,2]
    else:
        maxvalues = [maxval, maxval//2, maxval//4]

    for row in range(4):
        for column in range(4):
            val = newstate[row,column]
            currentval = newstate[row,column]
            if val != currentval: f3 += 1
            if val == 0: f4 += 1
            if val == maxval:
                # Check above
                if column-1 >= 0 and newstate[row,column-1] in maxvalues:
                    f4 = 1

                # Check right
                if row+1 <= 3 and newstate[row+1,column] in maxvalues:
                    f4 =  1

                # Check below
                if column+1 <= 3 and newstate[row,column+1] in maxvalues:
                    f4 =  1

                # Check left
                if row-1 >= 0 and newstate[row-1,column] in maxvalues:
                    f4 =  1
    return f2,f3,f4

''' F3 '''
# Sum of total changes 
def f3dep(individual, newstate):
    return 16-sum(individual.gameInstance.grid.flatten() == newstate.flatten())

''' F4 '''
# Number of non-empty cells in the given state of the board.
def f4dep(newstate):
    return sum([i for i in newstate.flatten() if i != 0])

''' F5'''
# Modified version of F5 : how many selected cells have equal neighbors.
def f5andf6(newstate,selected_cells=[[0,0],[0,3],[1,1],[2,2],[3,0]]):
    f5cells = [[0,0],[0,3],[1,1],[2,2],[3,0]]
    f6cells = [[2,3],[3,2],[0,1],[0,2],[1,1],[2,0],[1,3],[3,1]]
    f5counter = 0
    f6counter = 0
    for i in range(len(f6cells)):

        ### Checking f5 ###

        if i < len(f5cells):
            row = f5cells[i][0]
            column = f5cells[i][1]
            val = newstate[row,column] # Important: newstate can not be a list object
            # Check below
            if row+1 <= 3 and newstate[row+1,column] == val:
                f5counter += 1
            #Check the above
            if row-1 >= 0 and newstate[row-1,column] == val:
                f5counter += 1
            #Check left
            if column-1 >= 0 and newstate[row,column-1] == val:
                f5counter += 1
            #Check right
            if column+1 <= 3 and newstate[row,column+1] == val:
                f5counter +=1

        ### Checking f6 ###

        row = f6cells[i][0]
        column = f6cells[i][1]
        val = newstate[row,column] # Important: newstate can not be a list object
        # Check below
        if row+1 <= 3 and newstate[row+1,column] == val:
            f6counter += 1
        #Check the above
        if row-1 >= 0 and newstate[row-1,column] == val:
            f6counter += 1
        #Check left
        if column-1 >= 0 and newstate[row,column-1] == val:
            f6counter += 1
        #Check right
        if column+1 <= 3 and newstate[row,column+1] == val:
            f6counter +=1
    return f5counter, f6counter
    
''' F6 '''
# Check if the given cells have equal neighbors, reuse function f5_modified
def f6dep(newstate):
    selected_cells = [[2,3],[3,2],[0,1],[0,2],[1,1],[2,0],[1,3],[3,1]]
    return f5(newstate,selected_cells)





''' F7 '''
#1 if the row/column containing highest and 2nd highest value 
# tile does not have empty cell return 1
def f7andf8andf9(newstate, nzeros, newstateT):

    # Finding Highest value
    maxValue = max(newstate.flatten())
    nrow,ncolumn = np.argwhere(newstate==maxValue)[-1]

    # Second Highest Value
    secondMaxValue = max(np.where(newstate==maxValue, 1, newstate).flatten())
    if secondMaxValue == 1: nrow2,ncolumn2 = nrow,ncolumn
    else: nrow2,ncolumn2 = np.argwhere(newstate==secondMaxValue)[-1]

    ### f7 ### 
    if 0 in newstate[nrow] or 0 in newstateT[ncolumn] or 0 in newstate[nrow2] or 0 in newstateT[ncolumn2]: f7 = 0
    else: f7 = 1
    
    ### f8 ### 
    row,column = newstate[nrow], newstateT[ncolumn]
    if np.array_equiv(np.sort(row), row) or np.array_equiv(np.sort(row)[::-1], row) or np.array_equiv(np.sort(column), column) or np.array_equiv(np.sort(column)[::-1], column):
        f8 = 1
    else: f8 = 0

    ### f9 ### 
    adjacentrow = newstate[nrow -1  if nrow -1 >= 0 else nrow+1]
    rowtiledensity = (sum([1 for i in row if i != 0]) + sum([1 for i in adjacentrow if i != 0]))/(16-nzeros)
    adjacentcolumn = newstate[ncolumn -1  if ncolumn -1 >= 0 else ncolumn+1]
    coltiledensity = (sum([1 for i in column if i != 0]) + sum([1 for i in adjacentcolumn if i != 0]))/(16-nzeros)
    f9 = rowtiledensity+coltiledensity

    return f7,f8,f9


    
''' F8 '''
# If the row/column containing highest and 2nd highest value
# tile is in descending order, return 1
def f8(newstate):
    # Because the state will be modified during find 2nd hightest value
    # Create a copy to find the row/column of the 2nd highest value
    newstate_copy = newstate.copy()
    
    # Finding highest value
    maxValue = max(newstate.flatten())
    row,column = np.argwhere(newstate==maxValue)[-1]
    row,column = newstate[row], newstate.T[column]
    if np.array_equiv(np.sort(row), row) or np.array_equiv(np.sort(row)[::-1], row) or np.array_equiv(np.sort(column), column) or np.array_equiv(np.sort(column)[::-1], column):
        pass

    # Second Highest value
    if maxValue != 2: 
        secondMaxValue = max(np.where(newstate==maxValue, 1, newstate).flatten())
        if secondMaxValue == 1: return 0
        row,column = np.argwhere(newstate==secondMaxValue)[-1]
        row,column = newstate[row], newstate.T[column]
        if np.array_equiv(np.sort(row), row) or np.array_equiv(np.sort(row)[::-1], row) or np.array_equiv(np.sort(column), column) or np.array_equiv(np.sort(column)[::-1], column):
            return 1
    return 0
    
''' F9 '''
# Density around the highest tile cell
def f9(newstate):
    # Finding the highest and 2nd highest value
    newstate_copy = newstate.copy()
    
    # Finding highest value
    maxvalue = max(newstate.flatten())
    maxIndex = np.argmax(newstate) # Didn't consider more than one highest value
    max_row, max_column =maxIndex//4, maxIndex-(maxIndex//4)*4
    
    # Finding second highest value
    if maxvalue != 2:
        temp_state =np.where(newstate_copy == maxvalue, 1, newstate_copy)
        secondMax = max(temp_state.flatten())
        secondIndex = np.argmax(temp_state)
        second_row, second_column = secondMax//4, secondIndex-(secondIndex//4)*4
    else: return 0
    for row in range(4):
        for column in range(4):
            sum = 0 # Sum of tiles that doesn't in the row/column containing 1st and 2nd
            if row not in (max_row,second_row) and \
            column not in (max_column,second_column):
                sum +=newstate[row][column]
    # Sum of tiles in the row/column contains highest and 2nd highest value
    density_part1 = newstate.sum() - sum
    
    for row in range(4):
        for column in range(4):
            sum = 0
            if row not in (max_row+1,max_row-1,second_row+1,second_row-1)\
            and column not in (max_column+1,max_column-1,second_column+1,second_column-1):
                sum += newstate[row][column]
    density_part2 = (newstate.sum()-sum)/ newstate.sum()
    return density_part1 + density_part2



In [91]:
ind = toolbox.individual()
for i in range(0):
    ind.gameInstance.move(np.random.randint(0,4))
state = ind.gameInstance.project(0)

_,_, f4 = f2andf3andf4(ind.gameInstance.grid, state)


In [92]:
# The state evaluation function
def stateEvaluation(ind, state):
    flatstate = state.flatten()
    maxvalue = max(flatstate)
    stateT = state.T
    
    f2,f3,f4 = f2andf3andf4(ind.gameInstance.grid, state)
    f5,f6 = f5andf6(state)
    f7,f8,f9 = f7andf8andf9(state, f4, stateT)
    
    E = ind[0]*f1(state) + ind[1]*f2 + ind[2]*f3 + ind[3]*f4 + ind[4]*f5 + ind[5]*f6 + ind[6]*f7 + ind[7]*f8 + ind[8]*f9
    return E


In [98]:
ind = toolbox.individual()

statesEvaluation = []

# Generate possible future states
for move in moves:
    state = ind.gameInstance.project(move)
    E = stateEvaluation(ind, state)
    print(E)
    statesEvaluation.append(E)

chosenMove = moves[np.argmax(statesEvaluation)]


18.819781470398528
18.819781470398528
31.301124154675882
30.963487454271984


In [99]:
# Game Evaluation Function
def gameEvaluation(moves, individual, gameIterations):

    # Play game for 1000 moves
    for iter in range(gameIterations):
        statesEvaluation = []

        # Evaluate all possible moves (projections)
        for move in moves:
            state = ind.gameInstance.project(move)
            if np.array_equal(state, individual.gameInstance.grid): E = 0
            else: E = stateEvaluation(individual, state)
            statesEvaluation.append(E)
            
        # Choose the best possible move with maximum E
        chosenMove = moves[np.argmax(statesEvaluation)]

        try: individual.gameInstance.move(chosenMove)
        except: 
            fitness = (max(individual.gameInstance.grid.flatten()) + sum([i for i in individual.gameInstance.grid.flatten() if i == 0]),)
            individual.gameInstance.start()
            # Return fitness
            return fitness

    # Return the fitness, 
    # Maximum value and number of zeros after 1000 game moves
    fitness = (max(individual.gameInstance.grid.flatten()) + sum([i for i in individual.gameInstance.grid.flatten() if i == 0]),)
    individual.gameInstance.start()
    return fitness

In [95]:
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=mutation_probability)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("select", tools.selTournament, k=population_size)
toolbox.register("gameEvaluation", gameEvaluation, gameIterations=gameIterations, moves=moves)

In [96]:
def main():
    best = []
    # Create the population
    population = toolbox.population(population_size)

    # Evaluate
    for individual in population:
        individual.fitness.values = toolbox.gameEvaluation(individual=individual)

    # Keep track of the no. of generations
    generations = 0
    
    # Begin evolution
    while generations < 100:
                
        # Selection
        #selected = toolbox.select(population, len(population))
        offspring = [toolbox.clone(ind) for ind in population]
        
        # Crossover
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random()> crossover_probability:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        # Mutation
        for mutant in offspring:
            toolbox.mutate(mutant)
            del mutant.fitness.values

        # Evaluate new individuals with no fitness
        for individual in offspring:
            if not individual.fitness.valid:
                individual.fitness.values = toolbox.gameEvaluation(individual=individual)
   
        population = tools.selBest(offspring+population,k=2) + toolbox.select(offspring, tournsize=population_size-2)
        
        best.append(tools.selBest(population, k=1)[0])
        average = sum([ind.fitness.values[0] for ind in population])/len(population)
        print(best[-1].fitness.values[0], average, best[-1].gameInstance.highscore)
        
        # Increment the generation counter
        generations += 1    
    
    return tools.selBest(population,k=1)
    

In [97]:
run_algorithm = main()

16.0 16.0 16
16.0 16.0 16
16.0 16.0 16
16.0 16.0 16
16.0 16.0 16


KeyboardInterrupt: 

In [ ]:
best_individual = run_algorithm[0]

best_individual.gameInstance.start()
# Play game with best individual
for iter in range(100):
    statesEvaluation = []

    # Evaluate all possible moves (projections)
    for move in moves:
        state = best_individual.gameInstance.project(move)
        if np.array_equal(state, best_individual.gameInstance.grid): E = 0
        else: E = stateEvaluation(best_individual, state)
        statesEvaluation.append(E)
        
    # Choose the best possible move with maximum E
    chosenMove = moves[np.argmax(statesEvaluation)]
    try: best_individual.gameInstance.move(chosenMove)
    except: 
        print(best_individual.gameInstance.grid)
        break

print(best_individual.gameInstance.grid)
